### Imports

In [2]:
import math, requests, json

### Constants

In [31]:
# Constants
TWOMBA_ENABLED = True 
TAKER_FEE_CONSTANT = 0.00 # Updated from 0.07 - fee-less
CREATORS_EARN_WHOLE_FEE_UP_TO = 1000
MAX_CPMM_PROB = 0.99
MIN_CPMM_PROB = 0.01

# Flat fees
FLAT_TRADE_FEE = 0.1
FLAT_COMMENT_FEE = 1

DPM_PLATFORM_FEE = 0.0
DPM_CREATOR_FEE = 0.0
DPM_FEES = DPM_PLATFORM_FEE + DPM_CREATOR_FEE

# Math
EPSILON = 0.00000001

def floatingEqual(a: float, b: float, epsilon = EPSILON) -> bool:
    return abs(a - b) < epsilon

def floatingGreater(a: float, b: float, epsilon = EPSILON) -> bool:
    return a - epsilon > b

def floatingGreaterEqual(a: float, b: float, epsilon = EPSILON) -> bool:
    return a + epsilon >= b

def floatingLesserEqual(a: float, b: float, epsilon = EPSILON) -> bool:
    return a - epsilon <= b

def logInterpolation(min: float, max: float, value: float) -> float:
    if value <= min:
        return 0
    if value >= max:
        return 1
    
    return math.log(value - min + 1) / math.log(max - min + 1)

def logit(x: float):
    return math.log(x / (1 - x))


### Calculate Fees

In [32]:
class Fees:
    def __init__(self, creatorFee=0, platformFee=0, liquidityFee=0):
        self.creatorFee = creatorFee
        self.platformFee = platformFee
        self.liquidityFee = liquidityFee
    
    def __repr__(self):
        return f"Fees(creatorFee={self.creatorFee}, platformFee={self.platformFee}, liquidityFee={self.liquidityFee})"
    
    def add(self, other):
        return Fees(
            self.creatorFee + other.creatorFee,
            self.platformFee + other.platformFee,
            self.liquidityFee + other.liquidityFee
        )

def addObjects(obj1, obj2):
    return obj1.add(obj2)

def getTakerFee(shares: float, prob: float) -> float:
    return TAKER_FEE_CONSTANT * prob * (1 - prob) * shares

def getFeesSplit(totalFees, previouslyCollectedFees: Fees) -> Fees:
    if TWOMBA_ENABLED:
        return Fees(0, totalFees, 0)
    
    before1k = max(0, CREATORS_EARN_WHOLE_FEE_UP_TO - previouslyCollectedFees.creatorFee)
    allToCreatorAmount = min(totalFees, before1k)
    splitWithCreatorAmount = totalFees - allToCreatorAmount
    
    return Fees(
        creatorFee = allToCreatorAmount + splitWithCreatorAmount * 0.5,
        platformFee = splitWithCreatorAmount * 0.5,
        liquidityFee = 0
    )

def getFeeTotal(fees) -> float:
    return fees.creatorFee + fees.platformFee + fees.liquidityFee

def sumAllFees(fees_list) -> float:
    total_fees = Fees()
    for fee in fees_list:
        total_fees = addObjects(total_fees, fee)
    return getFeeTotal(total_fees)


### Calculate CPMM

In [33]:
class CpmmState:
    def __init__(self, pool: dict[str, float], p: float, collectedFees: Fees):
        self.pool = pool
        self.p = p
        self.collectedFees = collectedFees
        
def getCpmmProbability(pool: dict[str, float], p: float) -> float:
    YES, NO = pool['YES'], pool['NO']
    return (p * NO) / ((1 - p) * YES + p * NO)

def getCpmmProbabilityAfterBetBeforeFees(state: CpmmState, outcome: str, bet: float) -> float:
    pool, p = state.pool, state.p
    shares = calculateCpmmShares(pool, p, bet, outcome)
    YES, NO = pool['YES'], pool['NO']
    
    if outcome == "YES":
        newY, newN = YES - shares + bet, NO + bet
    else:
        newY, newN = YES + bet, NO - shares + bet
    
    return getCpmmProbability({'YES': newY, 'NO': newN}, p)

def getCpmmOutcomeProbabilityAfterBet(state: CpmmState, outcome: str, bet: float) -> float:
    newPool = calculateCpmmPurchase(state, bet, outcome)[1]
    p = getCpmmProbability(newPool, state.p)
    if outcome == 'NO':
        return 1 - p
    else:
        return p

# before liquidty fee
def calculateCpmmShares(pool: dict[str, float], p: float, betAmount: float, betChoice: str) -> float:
    if betAmount == 0:
        return 0
    
    YES, NO = pool['YES'], pool['NO']
    k = (YES ** p) * (NO ** (1 - p))
    
    # https://www.wolframalpha.com/input?i=%28y%2Bb-s%29%5E%28p%29*%28n%2Bb%29%5E%281-p%29+%3D+k%2C+solve+s
    if betChoice == 'YES':
        return YES + betAmount - (k * (betAmount + NO) ** (p - 1)) ** (1 / p)
    elif betChoice == 'NO':
        return NO + betAmount - (k * (betAmount + YES) ** -p) ** (1 / (1 - p))

def getCpmmFees(state: CpmmState, betAmount: float, outcome: str) -> tuple:
    fee = 0
    for _ in range(10):
        if betAmount == 0:
            break
        betAmountAfterFee = betAmount - fee
        shares = calculateCpmmShares(state.pool, state.p, betAmountAfterFee, outcome)
        averageProb = betAmountAfterFee / shares
        fee = getTakerFee(shares, averageProb)
    
    totalFees = 0 if betAmount == 0 else fee
    fees = getFeesSplit(totalFees, state.collectedFees)
    remainingBet = betAmount - totalFees
    
    return remainingBet, totalFees, fees

def calculateCpmmSharesAfterFee(state: CpmmState, bet: float, outcome: str) -> float:
    pool, p = state.pool, state.p
    remainingBet = getCpmmFees(state, bet, outcome)[0]
    
    return calculateCpmmShares(pool, p, remainingBet, outcome)

def calculateCpmmPurchase(state: CpmmState, bet: float, outcome: str, freeFees:bool=False) -> tuple:
    pool, p = state.pool, state.p
    remainingBet, fees = (bet, Fees()) if freeFees else (getCpmmFees(state, bet, outcome)[0], getCpmmFees(state, bet, outcome)[2])
    
    shares = calculateCpmmShares(pool, p, remainingBet, outcome)
    YES, NO = pool['YES'], pool['NO']
    
    fee = fees.liquidityFee 
    
    if outcome == 'YES':
        newY, newN = YES - shares + remainingBet + fee, NO + remainingBet + fee
    else:
        newY, newN = YES + remainingBet + fee, NO - shares + remainingBet + fee    
    
    postBetPool = {'YES': newY, 'NO': newN}

    newPool, newP = addCpmmLiquidity(postBetPool, p, fee)[0], addCpmmLiquidity(postBetPool, p, fee)[2]
    
    return shares, newPool, newP, fees

def calculateCpmmAmountToProb(state: CpmmState, prob: float, outcome: str) -> float:
    if prob <= 0 or prob >= 1 or math.isnan(prob):
        return float('inf')
    if outcome == 'NO':
        prob = 1 - prob
    
    pool, p = state.pool, state.p
    YES, NO = pool['YES'], pool['NO']
    k = YES ** p * NO ** (1 - p)
    # https://www.wolframalpha.com/input?i=-1+%2B+t+-+((-1+%2B+p)+t+(k%2F(n+%2B+b))^(1%2Fp))%2Fp+solve+b
    if outcome == 'YES':
        ((p * (prob - 1)) / ((p - 1) * prob)) ** -p * (k - NO * ((p * (prob - 1)) / ((p - 1) * prob)) ** p)
    else:
        (((1 - p) * (prob - 1)) / (-p * prob)) ** (p - 1) * (k - YES * (((1 - p) * (prob - 1)) / (-p * prob)) ** (1 - p))

def calculateCpmmAmountToProbIncludingFees(state: CpmmState, prob: float, outcome: str) -> float:
    amount = calculateCpmmAmountToProb(state, prob, outcome)
    shares = calculateCpmmShares(state.pool, state.p, amount, outcome)
    averageProb = amount / shares
    fees = getTakerFee(shares, averageProb)
    return amount + fees

def calculateCpmmAmountToBuySharesFixedP(state: CpmmState, shares: float, outcome: str) -> float:
    if not floatingEqual(state.p, 0.5):
        raise ValueError(f"calculateAmountToBuySharesFixedP only works for p = 0.5, got {state.p}")
    
    YES, NO = state.pool['YES'], state.pool['NO']
    # https://www.wolframalpha.com/input?i=%28y%2Bb-s%29%5E0.5+*+%28n%2Bb%29%5E0.5+%3D+y+%5E+0.5+*+n+%5E+0.5%2C+solve+b
    if outcome == 'YES':
        return (shares - YES - NO + math.sqrt(4 * NO * shares + (YES + NO - shares) ** 2)) / 2
    else:
        return (shares - YES - NO + math.sqrt(4 * YES * shares + (YES + NO - shares) ** 2)) / 2
    
# Faster version assuming p = 0.5
def calculateAmountToBuySharesFixedP(state: CpmmState, shares: float, outcome: str, unfilledBets):
    pass

def getCpmmLiquidity(pool: dict[str, float], p: float) -> float:
    YES, NO = pool['YES'], pool['NO']
    return (YES ** p) * (NO ** (1 - p))

def addCpmmLiquidity(pool: dict[str, float], p: float, amount: float) -> tuple:
    prob = getCpmmProbability(pool, p)
    
    # https://www.wolframalpha.com/input?i=p%28n%2Bb%29%2F%28%281-p%29%28y%2Bb%29%2Bp%28n%2Bb%29%29%3Dq%2C+solve+p
    YES, NO = pool['YES'], pool['NO']
    numerator = prob * (amount + YES)
    denominator = amount - NO * (prob - 1) + prob * YES
    newP = numerator / denominator
    newPool = {'YES': YES + amount, 'NO': NO + amount}
    
    oldLiquidity = getCpmmLiquidity(pool, newP)
    newLiquidity = getCpmmLiquidity(newPool, newP)
    liquidity = newLiquidity - oldLiquidity
    
    return newPool, liquidity, newP


### Calculate Multi CPMM

In [34]:
def getK(pool: dict[str, float]) -> float:
    values = pool.values()
    return sum(map(math.log, values))

def getLiquidity(pool: dict[str, float]):
    return math.exp(getK(pool) / len(pool))

### New Bet

In [35]:
def computeFill():
  pass

def computeFills():
  pass

def computeCpmmBet():
  pass

# limitProb

### Elasticity

In [9]:
ELASTICITY_BET_AMOUNT = 10_000 # readjust with platform volume

# totalLiquidity = ante

def computeBinaryCpmmElasticityFromAnte(ante: float, betAmount = ELASTICITY_BET_AMOUNT) -> float:
  pool = {'YES': ante, 'NO': ante}
  p = 0.5 # neutral probability
  state = CpmmState(pool, p, Fees())
  
  yesOutcome = calculateCpmmPurchase(state, betAmount, 'YES', freeFees=True)
  poolY, pY = yesOutcome[1], yesOutcome[2]
  resultYes = getCpmmProbability(poolY, pY)
  
  noOutcome = calculateCpmmPurchase(state, betAmount, 'NO', freeFees=True)
  poolN, pN = noOutcome[1], noOutcome[2]
  resultNo = getCpmmProbability(poolN, pN)
  
  return logit(resultYes) - logit(resultNo)  

def computeBinaryCpmmElasticity(state: CpmmState, betAmount: float = ELASTICITY_BET_AMOUNT) -> float:
  yesOutcome = calculateCpmmPurchase(state, betAmount, 'YES', freeFees=True)
  poolY, pY = yesOutcome[1], yesOutcome[2]
  resultYes = getCpmmProbability(poolY, pY)
  
  noOutcome = calculateCpmmPurchase(state, betAmount, 'NO', freeFees=True)
  poolN, pN = noOutcome[1], noOutcome[2]
  resultNo = getCpmmProbability(poolN, pN)
  
  resultYes = min(resultYes, 0.995)
  resultNo = max(resultNo, 0.005)
  
  return logit(resultYes) - logit(resultNo)  
  


### Test CPMM

In [10]:
pool = {'YES': 100, 'NO': 100}
p = 0.5
getCpmmProbability(pool, p)

0.5

In [11]:
pool = {'YES': 150, 'NO': 50}
p = 0.5
getCpmmProbability(pool, p)

0.25

In [12]:
pools = [{'YES': 1000000, 'NO': 1},
         {'YES': 1, 'NO': 1000000},
         {'YES': 1, 'NO': 100},
         {'YES': 1, 'NO': 100}]

p = [0.5, 0.5, 0.000001, 0.999999]

for i in range(len(pools)):
  prob = getCpmmProbability(pools[i], p[i])
  print(prob)
  print(prob > 0)
  print(prob < 1)
  

9.99999000001e-07
True
True
0.999999000001
True
True
9.999010098000296e-05
True
True
0.9999999899999902
True
True


In [13]:
pool = {'YES': 100, 'NO': 100}
p = 0.5
betAmount = 0
betChoice = 'YES'

calculateCpmmShares(pool, p, betAmount, betChoice)

0

In [14]:
pool = {'YES': 100, 'NO': 100}
p = 0.5
betAmount = 10
betChoice = 'YES'

calculateCpmmShares(pool, p, betAmount, betChoice)

19.090909090909065

In [15]:
pool = {'YES': 100, 'NO': 100}
p = 0.5
initialState = CpmmState(pool, p, Fees())

newPool = calculateCpmmPurchase(initialState, 5, 'YES')[1]

print(f'{newPool['YES'] = }')
print(f'{newPool['NO'] = }')

newPool['YES'] = 95.38819365450188
newPool['NO'] = 104.8347768930421


In [16]:
pool = {'YES': 100, 'NO': 100}
p = 0.5
initialState = CpmmState(pool, p, Fees())

newPool = calculateCpmmPurchase(initialState, 5, 'NO')[1]

print(f'{newPool['YES'] = }')
print(f'{newPool['NO'] = }')

newPool['YES'] = 104.8347768930421
newPool['NO'] = 95.38819365450188


In [17]:
pool = {'YES': 100.0, 'NO': 100.0}
p = 0.5
initialState = CpmmState(pool, p, Fees())

newPool = calculateCpmmPurchase(initialState, 10, 'NO')[1]

initalProduct = initialState.pool['YES'] * initialState.pool['NO']
newProduct = newPool['YES'] * newPool['NO']

print(initalProduct)
print(newProduct)
print(newPool)

10000.0
10000.0
{'YES': 109.67693839261675, 'NO': 91.17687042103996}


In [18]:
pool = {'YES': 100.0, 'NO': 100.0}
p = 0.5
initialState = CpmmState(pool, p, Fees())

newProb = getCpmmOutcomeProbabilityAfterBet(initialState, 'YES', 10)
print(newProb)

0.5460535652294758


In [19]:
pool = {'YES': 100.0, 'NO': 100.0}
p = 0.5
initialState = CpmmState(pool, p, Fees())

newProb = getCpmmOutcomeProbabilityAfterBet(initialState, 'NO', 10)
print(newProb) # probability of no
print(1 - newProb) # probability of yes

0.5460535652294758
0.4539464347705242


In [20]:
pool = {'YES': 100.0, 'NO': 100.0}
p = 0.5
initialState = CpmmState(pool, p, Fees())

newProb = getCpmmOutcomeProbabilityAfterBet(initialState, 'YES', 0)
print(newProb)

0.5


In [21]:
pool = {'YES': 100.0, 'NO': 100.0}
p = 0.5
amount = 10

cpmmLiquidity = addCpmmLiquidity(pool, p, amount)

newPool, liquidity = cpmmLiquidity[0], cpmmLiquidity[1]

print(newPool['YES'])
print(newPool['NO'])
print(liquidity)

110.0
110.0
10.0


In [22]:
pool = {'YES': 150.0, 'NO': 50.0}
p = 0.5
amount = 20

initialProb = getCpmmProbability(pool, p)
cpmmLiquidity = addCpmmLiquidity(pool, p, amount)
newPool, newP = cpmmLiquidity[0], cpmmLiquidity[2]
newProb = getCpmmProbability(newPool, newP)

print(initialProb)
print(newProb)

0.25
0.25000000000000006


### Test Cases

- **Manifold API**: https://docs.manifold.markets/api
- **GET request URL**: https://api.manifold.markets/v0
- **GET by market slug**: https://manifold.markets/api/v0/slug/
- **GET by market ID**: https://api.manifold.markets/v0/market/
- **GET bets**: https://api.manifold.markets/v0/bets/
- **p**: This is the probability around which the market liquidity is "centered'. Probability constant in $y^{p}n^{1-p} = k$

In [36]:
# Will Andrew Tate be found guilty of human (sex) trafficking?

id = "G6Lj2ccHZyp8aHzwVchb"
slug = "will-andrew-tate-be-found-guilty-of"

rooturl = "https://api.manifold.markets/v0/market/"
data = requests.get(rooturl + id).json()

n = data['pool']['NO']
y = data['pool']['YES']
prob = data['probability']
p = data['p']
liquidity = data['totalLiquidity']
volume = data['volume']

print(f'{n = }\n{y = }\n{prob = }\n{p = }\n{liquidity = }\n{volume = }')

k = (y**p) * (n**(1 - p))
print(f'{k = }') 

betsURL = "https://api.manifold.markets/v0/bets?contractId="
betsData = requests.get(betsURL+id).json()

creatorFee = 0
platformFee = 0
liquidityFee = 0
trades = []
for i in betsData:
    if i['isCancelled'] != None:
        trades.append(i)
    creatorFee += i['fees']['creatorFee']
    platformFee += i['fees']['platformFee']
    liquidityFee += i['fees']['liquidityFee']

print(f'Number of Bets = {len(trades)}')

print()
print(f'{creatorFee = }\n{platformFee = }\n{liquidityFee = }')

collectedFees = Fees(creatorFee, platformFee, liquidityFee)
pool = {'YES': y, 'NO': n}
state = CpmmState(pool, p, collectedFees)

# New Bet
bet = 1000
outcome = 'YES'
newProb = getCpmmOutcomeProbabilityAfterBet(state, outcome, bet)
newProb = newProb if outcome == 'YES' else 1 - newProb
totalFees = getCpmmFees(state, bet, outcome)[1]
maxPayout = calculateCpmmSharesAfterFee(state, bet, outcome)
print()
print(f"{newProb = }\n{totalFees = }\n{maxPayout = }")

print(computeBinaryCpmmElasticity(state, bet))

n = 11977.29597826031
y = 8120.5998670444205
prob = 0.6208553075162113
p = 0.5261190329786608
liquidity = 10000
volume = 2341292.737688623
k = 9762.5960566133
Number of Bets = 7478

creatorFee = 2521.000760571361
platformFee = 2301.104940551263
liquidityFee = 0

newProb = 0.65601718813302
totalFees = 0.0
maxPayout = 1565.843193626607
0.3974799624330091


In [24]:
# Will UniCredit successfully acquire a majority stake in Commerzbank by 2025?

id = "spqjzzszu4"
slug = "will-unicredit-successfully-acquire"

rooturl = "https://api.manifold.markets/v0/market/"
data = requests.get(rooturl + id).json()

n = data['pool']['NO']
y = data['pool']['YES']
prob = data['probability']
p = data['p']
liquidity = data['totalLiquidity']
volume = data['volume']

print(f'{n = }\n{y = }\n{prob = }\n{p = }\n{liquidity = }\n{volume = }')

k = (y**p) * (n**(1 - p))
print(f'{k = }')

betsURL = "https://api.manifold.markets/v0/bets?contractId="
betsData = requests.get(betsURL+id).json()

creatorFee = 0
platformFee = 0
liquidityFee = 0
trades = []
for i in betsData:
    if i['isCancelled'] != None:
        trades.append(i)
    creatorFee += i['fees']['creatorFee']
    platformFee += i['fees']['platformFee']
    liquidityFee += i['fees']['liquidityFee']

print(f'Number of Bets = {len(trades)}')

print()
print(f'{creatorFee = }\n{platformFee = }\n{liquidityFee = }')

collectedFees = Fees(creatorFee, platformFee, liquidityFee)
state = CpmmState({'YES': y, 'NO': n}, p, collectedFees)

# New Bet
bet = 100
outcome = 'NO'
newProb = getCpmmOutcomeProbabilityAfterBet(state, outcome, bet)
newProb = newProb if outcome == 'YES' else 1 - newProb
totalFees = getCpmmFees(state, bet, outcome)[1]
maxPayout = calculateCpmmSharesAfterFee(state, bet, outcome)
print()
print(f"{newProb = }\n{totalFees = }\n{maxPayout = }")

print(computeBinaryCpmmElasticity(state))

n = 1098.9304626378225
y = 909.975684539353
prob = 0.5470292697257112
p = 0.49999999999999994
liquidity = 1000
volume = 200
k = 999.9999999999999
Number of Bets = 3

creatorFee = 0
platformFee = 6.603704976775171
liquidityFee = 0

newProb = 0.4967850338088452
totalFees = 3.5249465770204016
maxPayout = 201.81490871065216
9.593052640139103


In [11]:
# Will the total number of Giving What We Can pledges made in 2024 be higher than in 2022?

id = "Y3lXD7s2CWFoRwHcnlm4"
slug = "will-the-total-number-of-giving-wha-cc4cca3a58ce"

rooturl = "https://api.manifold.markets/v0/market/"
data = requests.get(rooturl + id).json()

n = data['pool']['NO']
y = data['pool']['YES']
prob = data['probability']
p = data['p']
liquidity = data['totalLiquidity']
volume = data['volume']

print(f'{n = }\n{y = }\n{prob = }\n{p = }\n{liquidity = }\n{volume = }')

k = (y**p) * (n**(1 - p))
print(f'{k = }') 

betsURL = "https://api.manifold.markets/v0/bets?contractId="
betsData = requests.get(betsURL+id).json()

creatorFee = 0
platformFee = 0
liquidityFee = 0
trades = []
for i in betsData:
    if i['isCancelled'] != None:
        trades.append(i)
    creatorFee += i['fees']['creatorFee']
    platformFee += i['fees']['platformFee']
    liquidityFee += i['fees']['liquidityFee']

print(f'Number of Bets = {len(trades)}')

print()
print(f'{creatorFee = }\n{platformFee = }\n{liquidityFee = }')

collectedFees = Fees(creatorFee, platformFee, liquidityFee)
state = CpmmState({'YES': y, 'NO': n}, p, collectedFees)

# New Bet
bet = 1000
outcome = 'YES'
newProb = getCpmmOutcomeProbabilityAfterBet(state, outcome, bet)
newProb = newProb if outcome == 'YES' else 1 - newProb
totalFees = getCpmmFees(state, bet, outcome)[1]
maxPayout = calculateCpmmSharesAfterFee(state, bet, outcome)
print()
print(f"{newProb = }\n{totalFees = }\n{maxPayout = }")

print(computeBinaryCpmmElasticityFromAnte(liquidity))

n = 581.7660652638804
y = 2962.1948186564964
prob = 0.07337731030907553
p = 0.2873448976738495
liquidity = 1000
volume = 4589.108025636279
k = 928.6723240171632
Number of Bets = 57

creatorFee = 5.772602170741537
platformFee = 16.271986404137195
liquidityFee = 0

newProb = 0.6974213080555609
totalFees = 49.19456820301702
maxPayout = 3644.9057135526145
9.591581091193504


In [22]:
# Will Vladimir Putin attend the G20 in Brazil in 2024?

id = "fN12OYGS30DGubdZuKEC"
slug = "will-vladimir-putin-attend-the-g20-de36a6365070"

rooturl = "https://api.manifold.markets/v0/market/"
data = requests.get(rooturl + id).json()

n = data['pool']['NO']
y = data['pool']['YES']
prob = data['probability']
p = data['p']
liquidity = data['totalLiquidity']
volume = data['volume']

print(f'{n = }\n{y = }\n{prob = }\n{p = }\n{liquidity = }\n{volume = }')

k = (y**p) * (n**(1 - p))
print(f'{k = }')

betsURL = "https://api.manifold.markets/v0/bets?contractId="
betsData = requests.get(betsURL+id).json()

creatorFee = 0
platformFee = 0
liquidityFee = 0
trades = []
for i in betsData:
    if i['isCancelled'] != None:
        trades.append(i)
    creatorFee += i['fees']['creatorFee']
    platformFee += i['fees']['platformFee']
    liquidityFee += i['fees']['liquidityFee']

print(f'Number of Bets = {len(trades)}')

print()
print(f'{creatorFee = }\n{platformFee = }\n{liquidityFee = }')

collectedFees = Fees(creatorFee, platformFee, liquidityFee)
state = CpmmState({'YES': y, 'NO': n}, p, collectedFees)

# New Bet
bet = 1000
outcome = 'YES'
newProb = getCpmmOutcomeProbabilityAfterBet(state, outcome, bet)
newProb = newProb if outcome == 'YES' else 1 - newProb
totalFees = getCpmmFees(state, bet, outcome)[1]
maxPayout = calculateCpmmSharesAfterFee(state, bet, outcome)
print()
print(f"{newProb = }\n{totalFees = }\n{maxPayout = }")

print(computeBinaryCpmmElasticity(state))

n = 840.4294249568765
y = 1944.9930355709967
prob = 0.09109629043902893
p = 0.1882805892896765
liquidity = 1000
volume = 1948.9589602278047
k = 984.2691189518598
Number of Bets = 49

creatorFee = 6.122965415212506
platformFee = 3.9484740011031385
liquidityFee = 0

newProb = 0.8498559826700247
totalFees = 44.28421369780051
maxPayout = 2827.1044250438877
9.829687510292292


In [ ]:
# Get all my bets
import urllib.request
from prettytable import PrettyTable

url = "https://api.manifold.markets/v0/bets?username=Abdullah"
request = urllib.request.urlopen(url)

data = json.loads(request.read())

header =['id', 'amount', 'shares', 'outcome', 'probBefore', 'probAfter', ]

### Limit Order Tests

In [29]:
# Will Andrew Tate be found guilty of human (sex) trafficking?
# unfilled limit orders

url = "https://api.manifold.markets/v0/bets?contractId=G6Lj2ccHZyp8aHzwVchb&kinds=open-limit"

data = requests.get(url).json()
print(data)

[{'id': 'pQB0g1MxfVdA', 'fees': {'creatorFee': 0, 'platformFee': 0, 'liquidityFee': 0}, 'fills': [], 'isApi': False, 'amount': 0, 'shares': 0, 'userId': '70F1hyNBrNRaXOO3qJUm4WthH5r1', 'outcome': 'YES', 'isFilled': False, 'limitProb': 0.6, 'probAfter': 0.6185001356212757, 'contractId': 'G6Lj2ccHZyp8aHzwVchb', 'loanAmount': 0, 'probBefore': 0.6185001356212757, 'visibility': 'public', 'createdTime': 1727575285000, 'isCancelled': False, 'orderAmount': 100, 'isRedemption': False, 'betId': 'pQB0g1MxfVdA', 'updatedTime': 1727575285000}, {'id': '18GTZoz5QaUf', 'fees': {'creatorFee': 0, 'platformFee': 0, 'liquidityFee': 0}, 'fills': [], 'isApi': False, 'amount': 0, 'shares': 0, 'userId': 'YVzDfApmf5brx6rM55gTy2CTwTD3', 'outcome': 'YES', 'isFilled': False, 'limitProb': 0.58, 'probAfter': 0.6116628001271074, 'contractId': 'G6Lj2ccHZyp8aHzwVchb', 'loanAmount': 0, 'probBefore': 0.6116628001271074, 'visibility': 'public', 'createdTime': 1727529805000, 'isCancelled': False, 'orderAmount': 29, 'isRed